In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

def correlationMeasure(P, T):
    P = P.astype(np.float64)
    T = T.astype(np.float64)
    val = np.sum(P * T) / (np.sqrt(np.sum(P**2)) * np.sqrt(np.sum(T**2)))
    return val

def zeroMeanCorrelationMeasure(P, T):
    P = P.astype(np.float64)
    T = T.astype(np.float64)
    P_mean = np.mean(P)
    T_mean = np.mean(T)
    val = np.sum((P - P_mean) * (T - T_mean)) / (np.sqrt(np.sum((P - P_mean)**2)) * np.sqrt(np.sum((T - T_mean)**2)))
    return val

def sumOfSquaredDifferenceMeasure(P, T):
    P = P.astype(np.float64)
    T = T.astype(np.float64)
    val = np.sum((P - T)**2)
    return val

def normalizedCrossCorrelationMeasure(P, T):
    P = P.astype(np.float64)
    T = T.astype(np.float64)
    P_mean = np.mean(P)
    T_mean = np.mean(T)
    val = np.sum((P - P_mean) * (T - T_mean)) / (np.sqrt(np.sum((P - P_mean)**2)) * np.sqrt(np.sum((T - T_mean)**2)))
    return val

def template_matching(img, template, measure_func):
    h, w = template.shape
    M, N = img.shape
    a, b = (h - 1) // 2, (w - 1) // 2

    result = np.zeros((M, N))

    for i in range(a, M - a):
        for j in range(b, N - b):
            patch = img[i - a:i + a + 1, j - b:j + b + 1]
            result[i, j] = measure_func(patch, template)

    return result

# Load images
input_img = cv2.imread('input.png', cv2.IMREAD_GRAYSCALE)
template = cv2.imread('template.png', cv2.IMREAD_GRAYSCALE)

# Perform template matching
correlation_result = template_matching(input_img, template, correlationMeasure)
zero_mean_correlation_result = template_matching(input_img, template, zeroMeanCorrelationMeasure)
ssd_result = template_matching(input_img, template, sumOfSquaredDifferenceMeasure)
ncc_result = template_matching(input_img, template, normalizedCrossCorrelationMeasure)

# Find max/min values and their locations
_, max_val_corr, _, max_loc_corr = cv2.minMaxLoc(correlation_result)
_, max_val_zero_mean, _, max_loc_zero_mean = cv2.minMaxLoc(zero_mean_correlation_result)
min_val_ssd, _, min_loc_ssd, _ = cv2.minMaxLoc(ssd_result)
_, max_val_ncc, _, max_loc_ncc = cv2.minMaxLoc(ncc_result)

# Draw rectangles around the matching locations
input_img_color = cv2.imread('input.png')
h, w = template.shape
cv2.rectangle(input_img_color, max_loc_corr, (max_loc_corr[0] + w, max_loc_corr[1] + h), (255, 0, 0), 2)
cv2.rectangle(input_img_color, max_loc_zero_mean, (max_loc_zero_mean[0] + w, max_loc_zero_mean[1] + h), (0, 255, 0), 2)
cv2.rectangle(input_img_color, min_loc_ssd, (min_loc_ssd[0] + w, min_loc_ssd[1] + h), (0, 0, 255), 2)
cv2.rectangle(input_img_color, max_loc_ncc, (max_loc_ncc[0] + w, max_loc_ncc[1] + h), (255, 255, 0), 2)

# Display the input image with rectangles
cv2.imshow('Matching Results', input_img_color)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Display the result images
results = [correlation_result, zero_mean_correlation_result, ssd_result, ncc_result]
titles = ['Correlation', 'Zero-mean Correlation', 'Sum of Squared Difference', 'Normalized Cross Correlation']

for i, result in enumerate(results):
    plt.subplot(2, 2, i + 1)
    plt.imshow(result, cmap='jet')
    plt.title(titles[i])
    plt.xticks([]), plt.yticks([])

plt.show()
